# Wind Farm Test

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import time
import os
from IPython.display import display, clear_output
from scipy import optimize
from itertools import product

# Import custom libraries
# from import_model import Model
import functions as func
import visualization as viz

# Test libraries
# from import_model import CCM

In [234]:
# Import models
from floris.tools import FlorisInterface as floris

# Add names of newly added models in this list
model_names = [
    'CCM',
]

ref_models = [
    'LES',
]

# Functions
def create_farm_layout(
    D_rotor: float = 126,
    n_x: int = 1,
    n_y: int = 1,
    spacing_x: float = 5,
    spacing_y: float = 5,
    hexagonal: bool = False,
    spacing_hex: float = 5,
):
    if hexagonal:
        # Calculate the vertical and horizontal spacing between hexagon centers
        vertical_spacing = 0.5 * D_rotor * spacing_hex
        horizontal_spacing = np.sqrt(3) * D_rotor * spacing_hex

        # Lists to store the x and y coordinates of the grid points
        X = np.zeros([n_x, n_y])
        Y = np.zeros([n_x, n_y])

        # Generate the coordinates of the hexagonal grid
        for row in range(n_y):
            for col in range(n_x):
                x = col * horizontal_spacing
                y = row * vertical_spacing

                # Shift every other row horizontally by half the spacing
                if row % 2 == 1:
                    x += horizontal_spacing / 2
                
                X[col, row] = x
                Y[col, row] = y
    else:
        X, Y = np.meshgrid(
            D_rotor * spacing_x * np.arange(0, n_x, 1),
            D_rotor * spacing_y * np.arange(0, n_y, 1),
        )

    return X, Y

def create_uniform_angles(
        yaw: float, 
        tilt: float, 
        n_x: int, 
        n_y: int,
    ):
    yaw_angles = yaw * np.ones([n_x, n_y])
    tilt_angles = tilt * np.ones([n_x, n_y])

    return yaw_angles, tilt_angles

def create_farm_config(
    case: dict,
    extra_layers: int = 0,
):
    # Get number of turbines
    n_x = int(case['n_x'])
    n_y = int(case['n_y'])
    n_turbines = n_x * n_y

    # Initialize farm layout
    X = np.zeros((n_x, n_y + extra_layers))
    Y = np.zeros((n_x, n_y + extra_layers))
    yaw_angles = np.zeros((n_x, n_y + extra_layers))
    tilt_angles = np.zeros((n_x, n_y + extra_layers))

    # Get x and y positions of turbines
    if case['equal']:
        spacing_x = case['spacing_x'] * case['D_rotor']
        spacing_y = case['spacing_y'] * case['D_rotor']
        
        if case['hexagonal']:
            X[0, 0] = case['x_0']
            Y[0, 0] = case['y_0']

            X[1, 0] = X[0, 0] + func.cosd(30) * spacing_x
            Y[1, 0] = Y[0, 0] + func.sind(30) * spacing_y

            for i in range(2, n_x):
                X[i, 0] = X[i-2, 0] + func.cosd(30) * spacing_x * 2
                Y[i, 0] = Y[i-2, 0]

            for i in range(1, n_y + extra_layers):
                X[:, i] = X[:, 0]
                Y[:, i] = Y[:, i-1] + func.sind(30) * spacing_y * 2

        else:
            for i in range(n_x):
                for j in range(0, n_y + extra_layers):
                    X[i, j] = case['x_0'] + spacing_x * i
                    Y[i, j] = case['y_0'] + spacing_y * j

        for i in range(n_x):
            for j in range(0, n_y + extra_layers):
                yaw_angles[i, j] = case['yaw_0']
                tilt_angles[i, j] = case['tilt_0']
    else:
        for i in range(n_turbines):
            X[0, i] = case['x_' + str(i)]
            Y[0, i] = case['y_' + str(i)]
            yaw_angles[0, 0, i] = case['yaw_' + str(i)]
            tilt_angles[0, 0, i] = case['tilt_' + str(i)]

    # Create farm layout dictionary
    farm_config = {
        'wind_directions': np.array([case['wd']]),
        'wind_speeds': np.array([case['U_ref']]),
        'X': X,
        'Y': Y,
        'yaw_angles': yaw_angles,
        'tilt_angles': tilt_angles,
        'D_rotor': case['D_rotor'],
        'n_x': n_x,
        'n_y': n_y,
        'n_turbines': n_turbines,
    }

    return farm_config

def get_fitted_ABL(
    df_ref_flowfield: pd.DataFrame,
    ref_model: str,
    z_ref_guess: float = 100.,
    U_ref_guess: float = 10.,
    alpha_guess: float = 0.12,
):
    if ref_model not in ref_models:
        raise ValueError('"ref_model" is not in list of valid reference models')
    
    # # Add new reference model types here
    # if ref_model == 'LES':
    #     ref_model == LES()
    
    # # Get streamwise and spanwise velocity profile parameters
    # U_params, V_params = get_U_and_V_params(df_ref_flowfield)

    # # Get farm initialized with flow field equal to LES simulation
    # farm = get_farm_with_fitted_flow_field(
    #     U_params,
    #     V_params,
    # )

class LES:
    def __init__(
        self,
    ):
        pass




# Create classes for models
class CCM:
    '''
    If you want to update changes made in FLORIS, first move towards
    the right folder in the terminal with right environment by (for example):
    cd Documents\Technische Universiteit Eindhoven\Graduation Project
    And afterwards reinstall FLORIS as follows:
    pip install -e floris_tilt
    '''
    def __init__(
        self,
        model_params: dict = None,
        input_file: str = 'model_files/CCM/case_initial.yaml',
    ):
        self.input_file = input_file
        self.model_params = model_params

        self.farm = floris(self.input_file)
        
    def get_farm(self):
        return self.farm
    
    def set_model_params(
        self,
        model_params: dict,
    ):
        for key in model_params.keys():
            found = False

            if key in self.farm.floris.wake.wake_deflection_parameters['gaussm'].keys():
                self.farm.floris.wake.wake_deflection_parameters['gaussm'][key] = model_params[key]
                found = True

            if key in self.farm.floris.wake.wake_velocity_parameters['ccm'].keys():
                self.farm.floris.wake.wake_velocity_parameters['ccm'][key] = model_params[key]
                found = True
            
            if not found:
                print(f'Key named "{key}" not found in either model')
    
    def reinitialize_farm(
        self,
        farm_config: dict,
        model_params: dict = None,
    ):
        if model_params == None:
            model_params = self.model_params
        
        for _ in range(2):
            self.farm.reinitialize(
                layout_x=farm_config['X'].flatten(), 
                layout_y=farm_config['Y'].flatten(), 
                wind_directions=farm_config['wind_directions'],
                wind_speeds=farm_config['wind_speeds'],
            )

            if model_params is not None:
                self.set_model_params(model_params)

    def get_turbine_powers(
        self,
        farm_config: dict,
    ):  
        # Get yaw and tilt angles flattened and adjust for number of wind conditions
        yaw_angles = farm_config['yaw_angles'].flatten()[None, None]
        tilt_angles = farm_config['tilt_angles'].flatten()[None, None]

        farm_copy = self.farm.copy()

        # Calculate wakes
        farm_copy.calculate_wake(
            yaw_angles=yaw_angles,
            tilt_angles=tilt_angles,
        )

        # Get misalignment correction factors
        correction_factors = func.get_correction_factor_misalignment(
            yaw_angles,
            tilt_angles,
        )

        # Get total power (need to account for air density and correction factor)
        turbine_powers = farm_copy.get_turbine_powers() * \
            farm_copy.floris.flow_field.air_density * \
            correction_factors

        return turbine_powers

    def get_velocity_field(
        self,
        farm_config,
        coordinates,
    ):
        farm_copy = self.farm.copy()

        _, flowfield, _ = farm_copy.calculate_full_domain(
            x_bounds=coordinates['X'],
            y_bounds=coordinates['Y'],
            z_bounds=coordinates['Z'],
            yaw_angles=farm_config['yaw_angles'].flatten()[None, None],
            tilt_angles=farm_config['tilt_angles'].flatten()[None, None],
        )

        # Save velocities in velocity field
        velocity_field = {
            'U': flowfield.u_sorted[0, 0],
            'V': flowfield.v_sorted[0, 0],
            'W': flowfield.w_sorted[0, 0],
        }

        return velocity_field


# Create class to get model
class WakeModeling:
    def __init__(
        self,
        model_name: str,
        model_params: dict = None,
    ): 
        if model_name not in model_names:
            raise ValueError('"model_name" is not in list of valid model names')
        
        self.model_name = model_name
        self.model_params = model_params
        self.X = np.array([0])
        self.Y = np.array([0])

        self.set_model()

    def set_model(self):
        # Add new models right here
        if self.model_name == 'CCM':
            self.model = CCM(self.model_params)
            self.farm = self.model.get_farm()

    def get_farm(self):
        return self.farm
    
    def get_model_class(self):
        return self.model_class
    
    def set_farm_layout(
        self,
        D_rotor: float = 126,
        n_x: int = 1,
        n_y: int = 1,
        spacing_x: float = 5,
        spacing_y: float = 5,
        hexagonal: bool = False,
        spacing_hex: float = 5,
    ):
        self.X, self.Y = create_farm_layout(
            D_rotor=D_rotor, 
            n_x=n_x, 
            n_y=n_y, 
            spacing_x=spacing_x, 
            spacing_y=spacing_y, 
            hexagonal=hexagonal,
            spacing_hex=5, 
        )

    def set_farm_layout_custom(
        self,
        X,
        Y,
    ):
        self.X = X
        self.Y = Y
    
    def get_farm_layout(self):
        return self.X, self.Y

    def run_model(
        self,
        farm_config,
    ):
        self.model.reinitialize_farm(
            farm_config,
        )

        turbine_powers = self.model.get_turbine_powers(
            farm_config,
        )

        return turbine_powers

    def get_velocity_field(
        farm_config,
        coordinates,
    ):
        self.model.reinitialize_farm(
            farm_config,
        )

        velocity_field = self.model.get_velocity_field(
            farm_config,
            coordinates,
        )

        return velocity_field

### Load cases
Here, the csv file is read, which contains data about farm configurations.

In [61]:
# Specify location of csv and file name of csv
location_case_file = '../Optimization Framework/'
case_file_name = 'test_cases.csv'

# Read case file
df_loaded_cases = pd.read_csv(location_case_file + case_file_name)

# Get list of case names
mask_test = (df_loaded_cases['test'] == 1)
loaded_case_names = df_loaded_cases['case_name'][mask_test].reset_index(drop=True)

# Get number of cases
n_loaded_cases = len(loaded_case_names)

# Show begin of case file
df_loaded_cases.head()

,case_name,test,calibration_power_output,calibration_plane,optimization,stratisfaction_level,z_ref,U_ref,wd,wd_eff,...,yaw_0,tilt_0,x_1,y_1,yaw_1,tilt_1,x_2,y_2,yaw_2,tilt_2
0,1x1TURB_wd270_ws10_1x_y0_t5,1,NaN,NaN,1,neutral,100,10,270,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2x1TURB_wd270_ws10_2x_y0_t5,1,NaN,NaN,1,neutral,100,10,270,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3x1TURB_wd270_ws10_3x_y0_t5,1,NaN,NaN,1,neutral,100,10,270,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4x1TURB_wd270_ws10_4x_y0_t5,1,NaN,NaN,1,neutral,100,10,270,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2x2TURB_wd270_ws10_4x_y0_t5,1,NaN,NaN,1,neutral,100,10,270,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
# Specify location of reference data
location_ref_data = '../LES/'

# Get flowfield of LES to fit ABL
ref_case_name = '1TURB_wd270_ws10_1x_y0_t5'
ref_flowfield = pd.read_csv(location_ref_data + ref_case_name + '/' + ref_case_name + '.csv')

In [112]:
# Initialize dictionary of farm configurations
loaded_farm_configs = {}

# Create farm configurations
for idc, case_name in enumerate(loaded_case_names):
    loaded_farm_configs[case_name] = create_farm_config(
        df_loaded_cases[df_loaded_cases['case_name'] == case_name].iloc[0]
    )

### Create custom cases
Here, custom cases can be created.

In [113]:
custom_farm_configs = {}

In [190]:
new_custom_case_name = 'test'

n_x = 3
n_y = 3

X, Y = create_farm_layout(
    D_rotor=126,
    n_x=n_x,
    n_y=n_y,
    spacing_x=5,
    spacing_y=5,
    hexagonal=False,
    spacing_hex=5,
)

yaw_angles, tilt_angles = create_uniform_angles(
    yaw=0,
    tilt=5,
    n_x=n_x,
    n_y=n_y,
)

new_custom_case = {
    'wind_directions': [270],
    'wind_speeds': [10],
    'X': X,
    'Y': Y,
    'yaw_angles': yaw_angles,
    'tilt_angles': tilt_angles,
    'D_rotor': 126,
    'n_x': n_x,
    'n_y': n_y,
    'n_turbines': n_x * n_y
}

custom_farm_configs[new_custom_case_name] = new_custom_case

### Set wake modeling framework
Here, the model is specified, after which an instance of the wake modeling class is made.

In [221]:
# Set model name
model_name = 'CCM'

# Set model parameters
model_params = {
    'ad': 0,
    'bd': -0.0018192983887298023,
    'cd': 1.0803331806986867,
    'dd': -0.09040629347972164,
    'alpha': 0.58,
    'beta': 0.077,
    'dm': 1.0,
    'c_s1': 0.0563691592,
    'c_s2': 0.1376631233159683,
    'a_s': 0.3253111149080571,
    'b_s': 0.012031554853652504,
    'a_f': 3.11,
    'b_f': -0.68,
    'c_f': 2.223295807654856,
    'wr_gain': 0.5392489436318193,
    'ma_gain': 1.7431079762733077,
    'wr_decay_gain': 3.207532818500954,
    'ma_decay_gain': 1.7832719494462048,
}

# Get wakemodeling framework
wakemodeling = WakeModeling(
    model_name,
    model_params,
)

In [229]:
farm_configs = custom_farm_configs

case_names = list(farm_configs.keys())

turbine_powers = wakemodeling.run_model(
    farm_configs[case_name]
)

turbine_powers

_, flowfield, _ = wakemodeling.model.farm.copy().calculate_full_domain(
    x_bounds=np.linspace(0, 2000, 100),
    y_bounds=np.linspace(0, 2000, 100),
    z_bounds=np.linspace(1, 150, 100),
    yaw_angles=new_custom_case['yaw_angles'].flatten()[None, None],
    tilt_angles=new_custom_case['tilt_angles'].flatten()[None, None],
)

# Save velocities in velocity field
velocity_field = {
    'U': flowfield.u_sorted[0, 0],
    'V': flowfield.v_sorted[0, 0],
    'W': flowfield.w_sorted[0, 0],
}


In [233]:
velocity_field

{'U': array([[[ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848],
         [ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848],
         [ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848],
         ...,
         [ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848],
         [ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848],
         [ 5.82761562,  6.50652162,  6.88445454, ..., 10.60633641,
          10.61931005, 10.63216848]],
 
        [[ 5.19906465,  5.78165988,  6.05641431, ...,  7.60469104,
           7.69277132,  7.78673981],
         [ 5.33065223,  5.9268168 ,  6.21487125, ...,  7.78957297,
           7.8864338 ,  7.98867629],
         [ 5.60646438,  6.23913812,  6.56471418, ...,  8.43984496,
           8.55115864,  8.66534772],
         ...,
         [ 5.82761562,  6.50